In [1]:
import requests
import pandas as pd
import numpy as np
import pyodbc
import psycopg2
from datetime import datetime
from dotenv import load_dotenv
import os
import logging

In [2]:
# Configurar o logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


In [4]:
def clear_env_variables():
    """
    Limpa as variáveis de ambiente específicas usadas para conexão ao banco de dados.
    """
    keys_to_clear = ['SERVER_MIS', 'DATABASE_MIS', 'USERNAME_MIS', 'PASSWORD_MIS', 'DRIVER_MIS', 'PORT_MIS']
    for key in keys_to_clear:
        if key in os.environ:
            del os.environ[key]

In [5]:
def connect_to_database(env_file):
    """
    Carrega variáveis de ambiente de um arquivo .env especificado e estabelece uma conexão com o banco de dados.
    
    Args:
    env_file (str): Caminho para o arquivo .env.
    
    Returns:
    connection: Objeto de conexão com o banco de dados.
    cursor: Objeto cursor para interagir com o banco de dados.
    """

    # Limpar variáveis de ambiente específicas
    clear_env_variables()
    # Carregar variáveis de ambiente do arquivo .env especificado
    load_dotenv(env_file)
    
    # Parâmetros da conexão
    SERVER = os.getenv('SERVER_MIS')
    DATABASE = os.getenv('DATABASE_MIS')
    USERNAME = os.getenv('USERNAME_MIS')
    PASSWORD = os.getenv('PASSWORD_MIS')
    DRIVER = os.getenv('DRIVER_MIS')
    PORT = os.getenv('PORT_MIS') 
    
    # Logar as variáveis carregadas
    logging.info(f"Servidor: {SERVER}")
    logging.info(f"Banco de Dados: {DATABASE}")
    logging.info(f"Usuário: {USERNAME}")


    # Verificar se todas as variáveis de ambiente necessárias foram carregadas
    if not all([SERVER, DATABASE, USERNAME, PASSWORD, DRIVER]):
        raise ValueError("Certifique-se de que todas as variáveis de ambiente (SERVER_MIS, DATABASE_MIS, USERNAME_MIS, PASSWORD_MIS, DRIVER_MIS) estejam definidas no arquivo .env.")
    
    # Conexão com o SQL Server
    connection_string = f'DRIVER={DRIVER};SERVER={SERVER},{PORT};DATABASE={DATABASE};UID={USERNAME};PWD={PASSWORD}'
    connection = pyodbc.connect(connection_string)
    cursor = connection.cursor()
    
    return connection, cursor

"""
Exemplo de uso:

env_file_path = 'path/to/your/.env'  # Substitua pelo caminho para o seu arquivo .env

connection, cursor = connect_to_database(env_file_path)

Agora você pode usar 'connection' e 'cursor' para interagir com o banco de dados
"""


"\nExemplo de uso:\n\nenv_file_path = 'path/to/your/.env'  # Substitua pelo caminho para o seu arquivo .env\n\nconnection, cursor = connect_to_database(env_file_path)\n\nAgora você pode usar 'connection' e 'cursor' para interagir com o banco de dados\n"

In [6]:
def execute_query(cursor, query_file):
    """
    Executa uma consulta SQL a partir de um arquivo e retorna os resultados como DataFrame.
    
    Args:
    cursor: Objeto cursor para interagir com o banco de dados.
    query_file (str): Caminho para o arquivo SQL contendo a consulta.
    
    Returns:
    DataFrame: DataFrame contendo os resultados da consulta.
    """
    with open(query_file, 'r') as file:
        query = file.read()
    
    cursor.execute(query)
    columns = [column[0] for column in cursor.description]  # Obter nomes das colunas
    data = cursor.fetchall()  # Obter todas as linhas
    
    # Converter os dados para um DataFrame do Pandas
    df = pd.DataFrame(data, columns=columns)
    
    return df

In [7]:
def main(env_file, query_file):
    """
    Função principal para conectar ao banco, executar a consulta e salvar o resultado.
    
    Args:
    env_file (str): Caminho para o arquivo .env.
    query_file (str): Caminho para o arquivo SQL contendo a consulta.
    """
    # Conectar ao banco de dados
    connection, cursor = connect_to_database(env_file)
    logging.info("Conexão estabelecida com sucesso.")
    
    # Executar a consulta e obter o DataFrame
    df = execute_query(cursor, query_file)
    
    # Fechar a conexão
    cursor.close()
    connection.close()
    logging.info("Conexão fechada com sucesso.")
    
    return df

In [8]:
# Exemplo de uso
if __name__ == "__main__":
    env_files = [
        r'C:\Users\int.matheus\Desktop\DW\Data-Warehouse\Environment\.Env_07',
        r'C:\Users\int.matheus\Desktop\DW\Data-Warehouse\Environment\.Env_08'
    ]
    
    connections = []
    cursors = []

    for env_file_path in env_files:
        logging.info(f"Carregando variáveis de ambiente do arquivo: {env_file_path}")
        connection, cursor = connect_to_database(env_file_path)
        connections.append(connection)
        cursors.append(cursor)
        logging.info("Conexão estabelecida com sucesso.")
    
    # Executar consultas
    query = "SELECT CONTRATANTE_ID, FANTASIA FROM tbcontratante"
    dfs = []
    
    for cursor in cursors:
        cursor.execute(query)
        columns = [column[0] for column in cursor.description]  # Obter nomes das colunas
        data = cursor.fetchall()  # Obter todas as linhas
        result = [dict(zip(columns, row)) for row in data]
        df = pd.DataFrame(result)
        dfs.append(df)

    # Fechar as conexões
    for cursor, connection in zip(cursors, connections):
        cursor.close()
        connection.close()

    # Unir os DataFrames
    combined_df = pd.concat(dfs, ignore_index=True)

    # Exibir o DataFrame combinado
    display(combined_df)
    combined_df.to_excel('Combined_df.xlsx')

2024-06-21 11:36:20,181 - INFO - Carregando variáveis de ambiente do arquivo: C:\Users\int.matheus\Desktop\DW\Data-Warehouse\Environment\.Env_07
2024-06-21 11:36:20,184 - INFO - Servidor: 45.77.94.104
2024-06-21 11:36:20,184 - INFO - Banco de Dados: dbActyon_Tim_RPL
2024-06-21 11:36:20,184 - INFO - Usuário: mis
2024-06-21 11:36:21,391 - INFO - Conexão estabelecida com sucesso.
2024-06-21 11:36:21,391 - INFO - Carregando variáveis de ambiente do arquivo: C:\Users\int.matheus\Desktop\DW\Data-Warehouse\Environment\.Env_08
2024-06-21 11:36:21,391 - INFO - Servidor: 45.77.164.133
2024-06-21 11:36:21,391 - INFO - Banco de Dados: dbActyon_Kedu
2024-06-21 11:36:21,391 - INFO - Usuário: mis
2024-06-21 11:36:22,549 - INFO - Conexão estabelecida com sucesso.


,CONTRATANTE_ID,FANTASIA
0,1,TIM - CONTROLE / POS PAGO
1,2,TIM - ULTRA FIBRA / LIVE
2,3,TIM - OI
3,4,TIM - PILOTO DEEPDIVE
4,5,TIM - Baixo risco
5,6,TIM - Alto risco
6,1,KEDU
7,2,KEDU SAC
8,3,KEDU 2022
9,4,TESTE


In [37]:
# Carrega as variáveis de ambiente do arquivo .env
#load_dotenv(dotenv_path=r"C:\Users\Matheus Alexander\Desktop\DW\Data-Warehouse\Environment\.env")
# Exemplo de uso
if __name__ == "__main__":
    env_file_path = r'C:\Users\int.matheus\Desktop\DW\Data-Warehouse\Environment\.Env_07'  # Substitua pelo caminho para o seu arquivo .env
    connection, cursor = connect_to_database(env_file_path)
    logging.info("Conexão estabelecida com sucesso.")


if __name__ == "__main__":
    env_file_path = r'C:\Users\int.matheus\Desktop\DW\Data-Warehouse\Environment\.Env_08'  # Substitua pelo caminho para o seu arquivo .env
    connection, cursor = connect_to_database(env_file_path)
    logging.info("Conexão estabelecida com sucesso.")    

2024-06-21 09:40:35,419 - INFO - Servidor: 45.77.94.104
2024-06-21 09:40:35,419 - INFO - Banco de Dados: dbActyon_Tim_RPL
2024-06-21 09:40:35,419 - INFO - Usuário: mis
2024-06-21 09:40:35,529 - INFO - Conexão estabelecida com sucesso.
2024-06-21 09:40:35,550 - INFO - Servidor: 45.77.164.133
2024-06-21 09:40:35,550 - INFO - Banco de Dados: dbActyon_Kedu
2024-06-21 09:40:35,550 - INFO - Usuário: mis
2024-06-21 09:40:35,767 - INFO - Conexão estabelecida com sucesso.


In [38]:
data1 = cursor.execute("SELECT TOP 10 *  FROM tbcontratante")
#data = cursor.execute("SELECT TOP 10 *  FROM CallDetail")

data2 = cursor.execute("SELECT TOP 10 *  FROM tbcontratante")



In [ ]:
# Obter os dados
columns = [column[0] for column in cursor.description]  # Obter nomes das colunas
data = cursor.fetchall()  # Obter todas as linhas

# Converter os dados para uma lista de dicionários
result = [dict(zip(columns, row)) for row in data]

# Opcional: Converter os dados para um DataFrame do Pandas
df = pd.DataFrame(result)

# Exibir os resultados
display(df)

In [40]:
# Fechar a conexão
cursor.close()
connection.close()